In [1]:
import pandas as pd
import os

In [2]:
def saveDataFrame(dataframes,filepath,names):
    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    for df,sname in zip(dataframes,names):
        df.to_excel(writer,sheet_name=sname)   
    writer.save()

In [3]:
def calCulateBrick(base_area):
    bricks=round((base_area*32)/6,4)
    return bricks
def calculateConcreteMaterial(cem,ca,fa,block_vol):
    cem_w=cem*1440
    sand_w=ca*1600
    stone_w=fa*1700    
    sand_r=sand_w/cem_w
    stone_r=stone_w/cem_w
    water_r=0.45
    print("{}:{}:{}".format(1,sand_r, stone_r))
    cem_vol=1/(3150.0)
    sand_vol= sand_r/(2600.0)
    stone_vol=stone_r/(2700.0)
    water_vol=water_r/(1000.0)    
    un_compacted_volume_perkg=cem_vol+sand_vol+stone_vol+water_vol
    compacted_volume_perkg=un_compacted_volume_perkg*0.97
    print("un_compact_vol={} compact_vol={}".format(un_compacted_volume_perkg,compacted_volume_perkg))
    cement_in_weight=block_vol/compacted_volume_perkg
    sand_in_weight=cement_in_weight*sand_r
    stone_in_weight=cement_in_weight*stone_r
    print("in kg cement={} sand={} stone={}".format(cement_in_weight,sand_in_weight,stone_in_weight))
    sand_in_vol=sand_in_weight/1600.0
    stone_in_vol= stone_in_weight/1550.0
    myvalue={'cement':cement_in_weight,'sand':sand_in_vol,'stone':stone_in_vol}
    print(myvalue)
    return myvalue    
    
    
    

In [4]:
def calcualteFormWork(perimeter,height):
    h=height
    plain_sheet=round(perimeter*height,4)
    angle_length=2*perimeter+6*h
    angle1=(2*perimeter+6*h)*1.9157
    angle1=round(angle1,4)
    angle2=angle1/5855
    angle2=round(angle2,4)
    welding=int(angle_length*24)
    myvalue={'angle1':angle1,'angle2':angle2,'plain_sheet':plain_sheet,'welding':welding}
    print(myvalue)
    return myvalue

In [5]:
def calcualteQuantity(template,input_df,prod_data,cost_data):
    mylist=list(template['rate'])
    for index,row in input_df.iterrows():
        qmultiplirer=row['quantity_req']
        vat_rate=0.075
        code=row['codeNo']
        mydata=[]
        mycost_data=[]
        test_data
        L=row['Length']/100
        W=row['Bbreadth']/100
        H=row['Height']/100
        volume=L*W*H
        perimeter=2*(L+H)
        base_area=L*H
        print("volume={} perimeter={} base_area={}".format(volume,perimeter,base_area))
        """adding brick"""
        brick=calCulateBrick(base_area)
        
        "adding bricks"
        mydata.append(brick)
        mycost_data.append(brick*template.iloc[0,3])
       
        print("brick={}".format(brick))
        "adding sand,cement,stone"
        cem=row['Cement']
        sand=row['Sand']
        stone=row['Stone']
        myvalue=calculateConcreteMaterial(cem,sand,stone,volume)
        mydata.append(myvalue['cement'])
        mydata.append(myvalue['sand'])
        mydata.append(myvalue['stone'])
        mycost_data.append(myvalue['cement']*template.iloc[1,3])
        mycost_data.append(myvalue['sand']*template.iloc[2,3])
        mycost_data.append(myvalue['stone']*template.iloc[3,3])
        "adding disel and mobil"
        disel=prod_data.iloc[0,3]*volume
        mobil=prod_data.iloc[1,3]*volume
        print("disel={} mobil={}".format(disel,mobil))
        mydata.append(disel)
        mydata.append(mobil)
        mycost_data.append(disel*template.iloc[4,3])
        mycost_data.append(mobil*template.iloc[5,3])
        polythin=base_area*1.5
        """adding polythene"""
        mydata.append(polythin)
        mycost_data.append(polythin*template.iloc[6,3])
        print("polythin={}".format(polythin))
        
        putty=round(prod_data.iloc[2,3]*volume,4)
        print("putty={}".format(putty))
        """adding putty"""
        mydata.append(putty)
        mycost_data.append(putty*template.iloc[7,3])
        """Adding Metal Works"""
        formwork=calcualteFormWork(perimeter,H)
        mydata.append(formwork['angle1'])
        mydata.append(formwork['angle2'])
        mydata.append(formwork['plain_sheet'])
        mydata.append(formwork['welding'])
        mycost_data.append(formwork['angle1']*template.iloc[8,3]*0.015)
        mycost_data.append(formwork['angle2']*template.iloc[9,3]*0.015)
        mycost_data.append(formwork['plain_sheet']*template.iloc[10,3]*0.015)
        mycost_data.append(formwork['welding']*template.iloc[11,3]*0.015)
        """Adding Equipments"""
        mixture_machine=round(prod_data.iloc[3,3]*volume,4)
        mydata.append(mixture_machine)
        mycost_data.append(mixture_machine*template.iloc[12,3])
        """Adding Labours"""
        unskilled=round(prod_data.iloc[4,3]*volume,4)
        masson=round(prod_data.iloc[5,3]*volume,4)
        headmasson=round(prod_data.iloc[6,3]*volume,4)
        messonhelper=round(prod_data.iloc[7,3]*volume,4)
        operator=round(prod_data.iloc[8,3]*volume,4)
        welder=round(prod_data.iloc[9,3]*formwork['welding'],4)
        print("unsk={} m={} mhd={} mhl={} op={} welder={}".format(unskilled,masson,
                                                                  headmasson,messonhelper,operator,welder))
        mydata.append(unskilled)       
        mydata.append(masson)
        mydata.append(headmasson)
        mydata.append(messonhelper)
        mydata.append(operator)
        mydata.append(welder)
        
        mycost_data.append(unskilled*template.iloc[13,3])
        mycost_data.append(masson*template.iloc[14,3])        
        mycost_data.append(headmasson*template.iloc[15,3])
        mycost_data.append(messonhelper*template.iloc[16,3])
        mycost_data.append(operator*template.iloc[17,3])
        mycost_data.append(welder*template.iloc[18,3])     
        "trasfering analysis data to data frame"
        template[code]=mydata
        """Calcualting overhead,vat,tax"""
        sum_sub_total=sum(mycost_data)
        mycost_data.append(sum_sub_total)
        overhead=0.025*sum_sub_total
        mycost_data.append(overhead)
        item_total= sum_sub_total+overhead
        mycost_data.append(item_total)
        cp=item_total*0.1
        mycost_data.append(cp)
        total=item_total+cp
        mycost_data.append(total)
        vat=vat_rate*total
        mycost_data.append(vat)
        grand_total=total+vat
        mycost_data.append(grand_total)
        print(template.info())
        print(mydata)
        print(mycost_data)
        cost_data[code]=mycost_data
        myvalues={'q':template,'c':cost_data}
    return myvalues
def calcualteQuantity2(template,input_df,prod_data,cost_data):
    total_quantity_df=template.copy(deep=True)
    mylist=list(template['rate'])
    for index,row in input_df.iterrows():
        qmultiplirer=row['quantity_req']
        vat_rate=0.075
        code=row['codeNo']
        mydata=[]
        mycost_data=[]
        test_data=[]
        L=row['Length']/100
        W=row['Bbreadth']/100
        H=row['Height']/100
        volume=L*W*H
        perimeter=2*(L+H)
        base_area=L*H
        print("volume={} perimeter={} base_area={}".format(volume,perimeter,base_area))
        """adding brick"""
        brick=calCulateBrick(base_area)
        
        "adding bricks"
        mydata.append(brick)
        mycost_data.append(brick*template.iloc[0,3])
       
        print("brick={}".format(brick))
        "adding sand,cement,stone"
        cem=row['Cement']
        sand=row['Sand']
        stone=row['Stone']
        myvalue=calculateConcreteMaterial(cem,sand,stone,volume)
        mydata.append(myvalue['cement'])
        mydata.append(myvalue['sand'])
        mydata.append(myvalue['stone'])
        mycost_data.append(myvalue['cement']*template.iloc[1,3])
        mycost_data.append(myvalue['sand']*template.iloc[2,3])
        mycost_data.append(myvalue['stone']*template.iloc[3,3])
        "adding disel and mobil"
        disel=prod_data.iloc[0,3]*volume
        mobil=prod_data.iloc[1,3]*volume
        print("disel={} mobil={}".format(disel,mobil))
        mydata.append(disel)
        mydata.append(mobil)
        mycost_data.append(disel*template.iloc[4,3])
        mycost_data.append(mobil*template.iloc[5,3])
        polythin=base_area*1.5
        """adding polythene"""
        mydata.append(polythin)
        mycost_data.append(polythin*template.iloc[6,3])
        print("polythin={}".format(polythin))
        
        putty=round(prod_data.iloc[2,3]*volume,4)
        print("putty={}".format(putty))
        """adding putty"""
        mydata.append(putty)
        mycost_data.append(putty*template.iloc[7,3])
        """Adding Metal Works"""
        formwork=calcualteFormWork(perimeter,H)
        mydata.append(formwork['angle1'])
        mydata.append(formwork['angle2'])
        mydata.append(formwork['plain_sheet'])
        mydata.append(formwork['welding'])
        mycost_data.append(formwork['angle1']*template.iloc[8,3]*0.015)
        mycost_data.append(formwork['angle2']*template.iloc[9,3]*0.015)
        mycost_data.append(formwork['plain_sheet']*template.iloc[10,3]*0.015)
        mycost_data.append(formwork['welding']*template.iloc[11,3]*0.015)
        """Adding Equipments"""
        mixture_machine=round(prod_data.iloc[3,3]*volume,4)
        mydata.append(mixture_machine)
        mycost_data.append(mixture_machine*template.iloc[12,3])
        """Adding Labours"""
        unskilled=round(prod_data.iloc[4,3]*volume,4)
        masson=round(prod_data.iloc[5,3]*volume,4)
        headmasson=round(prod_data.iloc[6,3]*volume,4)
        messonhelper=round(prod_data.iloc[7,3]*volume,4)
        operator=round(prod_data.iloc[8,3]*volume,4)
        welder=round(prod_data.iloc[9,3]*formwork['welding'],4)
        print("unsk={} m={} mhd={} mhl={} op={} welder={}".format(unskilled,masson,
                                                                  headmasson,messonhelper,operator,welder))
        mydata.append(unskilled)       
        mydata.append(masson)
        mydata.append(headmasson)
        mydata.append(messonhelper)
        mydata.append(operator)
        mydata.append(welder)
        
        mycost_data.append(unskilled*template.iloc[13,3])
        mycost_data.append(masson*template.iloc[14,3])        
        mycost_data.append(headmasson*template.iloc[15,3])
        mycost_data.append(messonhelper*template.iloc[16,3])
        mycost_data.append(operator*template.iloc[17,3])
        mycost_data.append(welder*template.iloc[18,3])     
        "trasfering analysis data to data frame"
        template[code]=mydata
        """Calcualting overhead,vat,tax"""
        sum_sub_total=sum(mycost_data)
        mycost_data.append(sum_sub_total)
        overhead=0.025*sum_sub_total
        mycost_data.append(overhead)
        item_total= sum_sub_total+overhead
        mycost_data.append(item_total)
        cp=item_total*0.1
        mycost_data.append(cp)
        total=item_total+cp
        mycost_data.append(total)
        vat=vat_rate*total
        mycost_data.append(vat)
        grand_total=total+vat
        mycost_data.append(grand_total)
        print(template.info())
        print(mydata)
        print(mycost_data)
        cost_data[code]=mycost_data
        
        """Transfering Total quantity"""
        totalData=[x*qmultiplirer for x in mydata ]
        total_quantity_df[code]=totalData
        myvalues={'q':template,'c':cost_data,'tc': total_quantity_df}
    return myvalues

In [6]:
#myfolder=r'E:\Website_26_07_2020\cmis6\Civilworks cost\Analysis Template' #home computer
myfolder=r'F:\website\cmis6\Civilworks cost\Analysis Template' #Office computer
input_path=os.path.join(myfolder,'Material Analysis Input.xlsx')
output_path=os.path.join(myfolder,'Material Analysis output.xlsx')

In [7]:
sheetName='Template_Quantity'
q_template_df=pd.read_excel(input_path,sheet_name=sheetName)
sheetName='Production Rate'
production_rate_df=pd.read_excel(input_path,sheet_name=sheetName)
production_rate_df
sheetName='Block Data'
block_data_df=pd.read_excel(input_path,sheet_name=sheetName)
block_data_df
sheetName='Template_Cost'
c_template_df=pd.read_excel(input_path,sheet_name=sheetName)
"""Analysing data"""
myvalues=calcualteQuantity2(q_template_df,block_data_df,production_rate_df,c_template_df)
q_template_df=myvalues['q']
c_template_df=myvalues['c']
total_quantity_df=myvalues['tc']



volume=0.125 perimeter=2.0 base_area=0.25
brick=1.3333
1:2.7777777777777777:5.902777777777778
un_compact_vol=0.004022050377605933 compact_vol=0.003901388866277755
in kg cement=32.03987202620493 sand=88.99964451723592 stone=189.12424459912634
{'cement': 32.03987202620493, 'sand': 0.05562477782327245, 'stone': 0.1220156416768557}
disel=0.1765375 mobil=0.00885
polythin=0.375
putty=0.0041
{'angle1': 13.4099, 'angle2': 0.0023, 'plain_sheet': 1.0, 'welding': 168}
unsk=0.3578 m=0.0232 mhd=0.0043 mhl=0.0109 op=0.0074 welder=0.0028
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   codeno       19 non-null     object 
 1   Description  19 non-null     object 
 2   Unit         19 non-null     object 
 3   rate         19 non-null     float64
 4   40-190-15    19 non-null     float64
dtypes: float64(2), object(3)
memory usage: 888.0+ bytes
None
[1.3333, 32.0

In [8]:
myframes=[]
mynames=[]
myframes.append(q_template_df)
mynames.append('uint quantity')
myframes.append(total_quantity_df)
mynames.append('total quantity')
myframes.append(production_rate_df)
mynames.append('production_rate')
myframes.append(block_data_df)
mynames.append('analysis_list')
myframes.append(c_template_df)
mynames.append('Analyzed_cost')
saveDataFrame(myframes,output_path,mynames)

In [9]:
production_rate_df

,code,Description,unit,consumption/unit volume,Production /unit
0,20-245,Diesel/ Fuel,litr/cum,1.412300,0.708100
1,20-310,Lubricant ( Mobile oil etc.),litr/cum,0.070800,14.124300
2,20-505,Putty,kg/cum,0.032900,30.395137
3,70-080,Mixture Machine 0.2 cum,day/cum,0.059300,16.863406
4,80-375,"Labour,Unskilled",day/cum,2.862000,0.349406
5,80-425,Mason,day/cum,0.186000,5.376344
6,80-430,"Mason, Head",day/cum,0.034600,28.901734
7,80-435,"Mason, Helper",day/cum,0.087200,11.467890
8,80-500,"Operator, Mixture Machine",day/cum,0.059300,16.863406
9,80-755,Welder,day/point,0.000017,59545.454545


In [10]:
c_template_df

,codeno,Description,Unit,rate,40-190-15,40-190-05,40-190-35,40-190-40,28-120-20
0,10-040,Bricks : 1st Class,Nos,9.5,12.666350,7.600000,8.106350,4.053650,50.666350
1,10-100,Cement : Portland .,Kgs,10.0,320.398720,192.239232,164.044145,82.022072,3789.877197
2,10-380,"Sand, FM >= 1.5",cum,1050.0,58.406017,35.043610,29.903881,14.951940,414.517818
3,10-475,"Stone, Chips : 40mm down graded",cum,6500.0,793.101671,475.861003,406.068056,203.034028,5628.782665
4,20-245,Diesel/ Fuel,litre,65.0,11.474938,6.884963,5.875168,2.937584,91.799500
5,20-310,Lubricant ( Mobile oil etc.),litre,400.0,3.540000,2.124000,1.812480,0.906240,28.320000
6,20-500,Polythene Sheet : (1.0 kg per 16.00 sqm),sqm,15.0,5.625000,3.375000,3.600000,1.800000,22.500000
7,20-505,Putty,kg,80.0,0.328000,0.200000,0.168000,0.088000,2.632000
8,40-020,"Angle, M.S",kg,60.0,12.068910,8.620650,9.655110,6.206850,24.137820
9,40-020,"(Angle, M.S. .... ... ... ... ... ... ...)",kg,60.0,0.002070,0.001440,0.001620,0.001080,0.004140


In [11]:
q_template_df

,codeno,Description,Unit,rate,40-190-15,40-190-05,40-190-35,40-190-40,28-120-20
0,10-040,Bricks : 1st Class,Nos,9.5,1.333300,0.800000,0.853300,0.426700,5.333300
1,10-100,Cement : Portland .,Kgs,10.0,32.039872,19.223923,16.404414,8.202207,378.987720
2,10-380,"Sand, FM >= 1.5",cum,1050.0,0.055625,0.033375,0.028480,0.014240,0.394779
3,10-475,"Stone, Chips : 40mm down graded",cum,6500.0,0.122016,0.073209,0.062472,0.031236,0.865967
4,20-245,Diesel/ Fuel,litre,65.0,0.176538,0.105923,0.090387,0.045194,1.412300
5,20-310,Lubricant ( Mobile oil etc.),litre,400.0,0.008850,0.005310,0.004531,0.002266,0.070800
6,20-500,Polythene Sheet : (1.0 kg per 16.00 sqm),sqm,15.0,0.375000,0.225000,0.240000,0.120000,1.500000
7,20-505,Putty,kg,80.0,0.004100,0.002500,0.002100,0.001100,0.032900
8,40-020,"Angle, M.S",kg,60.0,13.409900,9.578500,10.727900,6.896500,26.819800
9,40-020,"(Angle, M.S. .... ... ... ... ... ... ...)",kg,60.0,0.002300,0.001600,0.001800,0.001200,0.004600
